<a href="https://colab.research.google.com/github/goldjunge3010/masterpraktikum/blob/main/Demo_custom_plots_in_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Installation and imports

In [17]:
!python -m pip install -q wandb

# import necessary packages
import numpy as np
import pandas as pd
import tensorflow as tf
import re

import wandb
from wandb.keras import WandbCallback
from wandb.keras import WandbMetricsLogger
import wandb.apis.reports as wr

# function to count sequence length
def count_seq_length(df: pd.DataFrame, seq_col: str) -> pd.Series:
    pattern = re.compile(r"\[UNIMOD:.*\]", re.IGNORECASE)
    df[seq_col].replace(pattern, "", inplace= True)
    return df[seq_col].str.len()

# 1. Count sequence lengths

In [18]:
# read data
TRAIN_DATAPATH = 'https://raw.githubusercontent.com/goldjunge3010/masterpraktikum/main/third_pool_tresh_1_0_train.csv'
train_set = pd.read_csv(TRAIN_DATAPATH)
# count sequence lengths
counts = count_seq_length(pd.read_csv(TRAIN_DATAPATH), "modified_sequence")
# ensure that 'counts' is a dataframe
counts_df = counts.to_frame()

# 2. Initialize Weights and Biases

In [19]:
# Initialize WANDB
PROJECT = 'demo_custom_plots'
RUN = "run_1"
wandb.init(project = PROJECT, name = RUN)
# create wandb table
table = wandb.Table(dataframe = counts_df)
# create custom plot
hist = wandb.plot_table(
  vega_spec_name="master_praktikum/hist_pep_len",
  data_table = table,
  fields = {"value" : 'modified_sequence'}
)
wandb.log({"Histogram": hist})
# finish run
wandb.finish()

# 3. Create report

In [20]:
# create report
report = wr.Report(
    project = PROJECT,
    title = "Working custom plots in report!",
    description = "The result of pain, blood and tears."
)
report.save()

Report(project='demo_custom_plots', entity='master_praktikum', title='Working custom plots in report!', description='The result of pain, blood and tears.', width='readable')

# 4. Adding custom plot to report

In [21]:
custom_plot = wr.PanelGrid(
    panels = [
      wr.CustomChart(
        query = {'summaryTable': {"tableKey" : "Histogram_table"}},
        chart_name='master_praktikum/hist_pep_len',
        chart_fields={'value': 'modified_sequence'}
      )
    ]
)

report.blocks = [wr.H1("Working custom plot"), custom_plot]

report.save()

Report(project='demo_custom_plots', entity='master_praktikum', title='Working custom plots in report!', description='The result of pain, blood and tears.', width='readable', blocks=[H1(text=['Working custom plot']), PanelGrid(runsets=[Runset(entity='master_praktikum', project='demo_custom_plots', name='Run set', query='', filters={'$or': [{'$and': []}]}, order=['-CreatedTimestamp'])], panels=[CustomChart(query={'runSets': {'runSets': '${runSets}', 'limit': 500}}, chart_name='master_praktikum/hist_pep_len', chart_fields={'value': 'modified_sequence'})])])